In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import env
import wrangle

pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [21]:
url = env.get_db_url('superstore_db')

ssdb = pd.read_sql('''
SELECT *
FROM orders
JOIN categories
USING(`Category ID`)
JOIN customers
USING(`Customer ID`)
JOIN products
USING(`Product ID`)
JOIN regions
USING(`Region ID`)
''', url)

In [22]:
ssdb.to_csv('ssdb.csv')

In [23]:
df = ssdb

In [24]:
#shape of the original dataset
df.shape

(1734, 22)

In [25]:
df.head().T

,0,1,2,3,4
Region ID,1,1,1,1,1
Product ID,FUR-BO-10001798,FUR-CH-10000454,FUR-TA-10000577,OFF-ST-10000760,OFF-PA-10002365
Customer ID,CG-12520,CG-12520,SO-20335,SO-20335,AA-10480
Category ID,1,2,4,5,11
Order ID,CA-2016-152156,CA-2016-152156,US-2015-108966,US-2015-108966,CA-2017-114412
Order Date,2016-11-08,2016-11-08,2015-10-11,2015-10-11,2017-04-15
Ship Date,2016-11-11,2016-11-11,2015-10-18,2015-10-18,2017-04-20
Ship Mode,Second Class,Second Class,Standard Class,Standard Class,Standard Class
Segment,Consumer,Consumer,Consumer,Consumer,Consumer
Country,United States,United States,United States,United States,United States


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1734 entries, 0 to 1733
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Region ID      1734 non-null   int64  
 1   Product ID     1734 non-null   object 
 2   Customer ID    1734 non-null   object 
 3   Category ID    1734 non-null   int64  
 4   Order ID       1734 non-null   object 
 5   Order Date     1734 non-null   object 
 6   Ship Date      1734 non-null   object 
 7   Ship Mode      1734 non-null   object 
 8   Segment        1734 non-null   object 
 9   Country        1734 non-null   object 
 10  City           1734 non-null   object 
 11  State          1734 non-null   object 
 12  Postal Code    1734 non-null   float64
 13  Sales          1734 non-null   float64
 14  Quantity       1734 non-null   float64
 15  Discount       1734 non-null   float64
 16  Profit         1734 non-null   float64
 17  Category       1734 non-null   object 
 18  Sub-Cate

## Observations for Prep

- Columns to be removed (unnecesary for our goal)
 0   region_id      1734 non-null   int64  
 1   product_id     1734 non-null   object 
 3   category_id    1734 non-null   int64  
 
- Column name formatting for ease of exploration and potential modelling
make all columns lower case 
replace all column name spaces with underscore

- Set date columns to datetime type
order_date
ship_date

- Set postal code to object type for ease of exploration and potential modelling

- Set order_date as index and sort by index for time series analysis

- Engineer column for the number of days it takes to ship from the order date for potential exploration



    # Set date columns to datetime type
    df.order_date = pd.to_datetime(df.order_date)
    df.ship_date = pd.to_datetime(df.ship_date)

    # Set order date as index
    df = df.set_index('order_date').sort_index()

    # Create column for number of days to ship
    df['days_to_ship'] = df['ship_date'] - df.index
    
    # Set postal code to object type
    df.postal_code = df.postal_code.astype('object')

In [6]:
df = rw.prep_superstore(df)

In [7]:
#shape of the prepared dataset
df.shape

(1734, 18)

In [8]:
df.isnull().sum()

order_id         0
ship_date        0
ship_mode        0
segment          0
country          0
city             0
state            0
postal_code      0
sales            0
quantity         0
discount         0
profit           0
category         0
sub-category     0
customer_name    0
product_name     0
region_name      0
days_to_ship     0
dtype: int64

In [9]:
df.describe()

,postal_code,sales,quantity,discount,profit,days_to_ship
count,1734.000000,1734.000000,1734.000000,1734.000000,1734.000000,1734.000000
mean,53501.926182,229.206358,3.794694,0.160110,21.880351,-0.069204
std,32180.245224,532.862614,2.297407,0.206419,202.891949,9.997567
min,1841.000000,1.080000,1.000000,0.000000,-3839.990400,-29.000000
25%,19143.000000,16.695000,2.000000,0.000000,1.702400,2.000000
50%,50315.000000,52.245000,3.000000,0.200000,8.073050,4.000000
75%,85705.000000,209.917500,5.000000,0.200000,27.414450,5.000000
max,98661.000000,8159.952000,14.000000,0.800000,3177.475000,7.000000


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1734 entries, 2014-01-04 to 2017-12-30
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_id       1734 non-null   object        
 1   ship_date      1734 non-null   datetime64[ns]
 2   ship_mode      1734 non-null   object        
 3   segment        1734 non-null   object        
 4   country        1734 non-null   object        
 5   city           1734 non-null   object        
 6   state          1734 non-null   object        
 7   postal_code    1734 non-null   float64       
 8   sales          1734 non-null   float64       
 9   quantity       1734 non-null   float64       
 10  discount       1734 non-null   float64       
 11  profit         1734 non-null   float64       
 12  category       1734 non-null   object        
 13  sub-category   1734 non-null   object        
 14  customer_name  1734 non-null   object        
 15  pro

In [11]:
df.head()

,order_id,ship_date,ship_mode,segment,country,city,state,postal_code,sales,quantity,discount,profit,category,sub-category,customer_name,product_name,region_name,days_to_ship
order_date,,,,,,,,,,,,,,,,,,
2014-01-04,CA-2014-112326,2014-01-08,Standard Class,Home Office,United States,Naperville,Illinois,60540.0,11.784,3.0,0.2,4.2717,Office Supplies,Labels,Phillina Ober,Avery 508,Central,4
2014-01-04,CA-2014-112326,2014-01-08,Standard Class,Home Office,United States,Naperville,Illinois,60540.0,272.736,3.0,0.2,-64.7748,Office Supplies,Storage,Phillina Ober,SAFCO Boltless Steel Shelving,Central,4
2014-01-04,CA-2014-112326,2014-01-08,Standard Class,Home Office,United States,Naperville,Illinois,60540.0,3.540,2.0,0.8,-5.4870,Office Supplies,Binders,Phillina Ober,GBC Standard Plastic Binding Systems Combs,Central,4
2014-01-09,CA-2014-135405,2014-01-13,Standard Class,Consumer,United States,Laredo,Texas,78041.0,31.200,3.0,0.2,9.7500,Technology,Accessories,Melanie Seite,Memorex Micro Travel Drive 8 GB,Central,4
2014-01-09,CA-2014-135405,2014-01-13,Standard Class,Consumer,United States,Laredo,Texas,78041.0,9.344,2.0,0.2,1.1680,Office Supplies,Art,Melanie Seite,Newell 312,Central,4


In [12]:
df.region_name.unique()

array(['Central', 'South', 'East', 'West'], dtype=object)